<a href="https://colab.research.google.com/github/IsaacFigNewton/DAG-Based-Compression/blob/main/DAG_Encoding.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install packages and import libraries

In [1]:
# !pip install --upgrade ete3  # http://etetoolkit.org/docs/latest/reference/reference_treeview.html#treestyle
# !pip install PyQt5

# import os
# # Set the environment variables for PyQt5
# os.environ['QT_QPA_PLATFORM_PLUGIN_PATH'] = '/usr/lib/x86_64-linux-gnu/qt5/plugins'
# os.environ['QT_PLUGIN_PATH'] = '/usr/lib/x86_64-linux-gnu/qt5/plugins'
# os.environ['QT_QPA_PLATFORM'] = 'xcb'
# os.environ['QT_DEBUG_PLUGINS'] = '1'

# # Keep trying to get this working to make things look nice
# import ete3
# from ete3 import Tree, TreeStyle

In [2]:
import numpy as np
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

import warnings
# warnings.filterwarnings('ignore')

# Classes

In [34]:
class SuffixNode:
  def __init__ (self, suffix=None, children=None, parent=None):
    self.suffix = suffix
    if children is None:
      children = dict()
    self.children = children
    self.parent = parent

  def __str__(self):
    return self.suffix

  def add_suffix(self, suffix):
    # if the suffix is empty, return
    if not suffix:
        return

    # Find the longest prefix match in the children
    for child_key, child_node in self.children.items():
        # Find the index of the longest common prefix
        i = 0
        while i < len(child_key) and i < len(suffix) and child_key[i] == suffix[i]:
            i += 1

        # If there is a common prefix
        if i > 0:
            if i == len(child_key):
                # If the entire child key is a prefix, recurse into that child
                child_node.add_suffix(suffix[i:])

            elif i < len(suffix):
                # Split the edge
                old_suffix = child_key[i:]
                new_suffix = suffix[i:]

                # Replace the old entry for the current node with a new one for the edge split
                #   ie AGG ==> AG
                #               |
                #               G
                self.children[child_key[:i]] = SuffixNode(suffix=child_key[:i],
                                                          parent=self,
                                                          children=child_node.children)
                del self.children[child_key]
                child_node = self.children[child_key[:i]]
                child_key = child_key[:i]

                # Create a new node for the existing edge suffix
                split_node = SuffixNode(suffix=old_suffix, parent=child_node)
                split_node.children = child_node.children

                # Create a new node for the new suffix
                child_node.suffix = child_key[:i]
                child_node.children = {old_suffix: split_node}
                child_node.children[new_suffix] = SuffixNode(suffix=new_suffix, parent=child_node)

            return

    # No matching prefix, add the suffix as a new child
    self.children[suffix] = SuffixNode(suffix=suffix, parent=self)

In [35]:
def print_tree(tree, indent = 0):
    # Iterate over the keys (features) in the tree
    for key, node in tree.children.items():
        print(' ' * indent + str(key))
        # If the node is a SuffixNode, recursively print the subtree
        if isinstance(node, SuffixNode):
            print_tree(node, indent + 4)
        else:
            print(' ' * (indent + 4) + str(key))

#Build Suffix Tree

In [38]:
test = "CAGTCAGG"

root = SuffixNode()
root = SuffixNode(children = {
    "C": SuffixNode(),
    "A": SuffixNode(),
    "T": SuffixNode(),
    "G": SuffixNode()
})

# loop through the string, starting with the last character
for i in range(0, len(test)):
  suffix = test[len(test) - i - 1:]

  # add the suffix to the tree
  root.add_suffix(suffix)


In [39]:
print_tree(root)

C
    AG
        G
        TCAGG
A
    G
        G
        TCAGG
T
    CAGG
G
    G
    TCAGG


# To fix later

In [7]:
# # Loads a tree structure from a newick string. The returned variable ’t’ is the root node for the tree.
# t = Tree("(A:1,(B:1,(E:1,D:1):0.5):0.5);" )

# print(t)

# # ts = TreeStyle()
# # ts.show_leaf_name = True

# # t.show(tree_style=ts)